## Project: Delta Lake for Movie Jsons

In [ ]:
# path
import re
source_path = '/mnt/antrastrg01adls/adlscontainer01/dataset/'
raw_path = '/FileStore/movie/raw/'
bronze_path = '/FileStore/movie/bronze/'
silver_path = '/FileStore/movie/silver/'
gold_path = '/FileStore/movie/gold/'

### 0. mounted storage to a Raw folder

In [ ]:
# Make Raw Idempotent
dbutils.fs.rm(raw_path, recurse=True)

def retrieve_data(file: str, dest_path: str) -> bool:
    """Download file from remote location to driver. Move from driver to DBFS."""
    src = source_path + file
    dest = dest_path + file
    dbutils.fs.cp(src, dest)
    return True

def prepare_raw(dest_path=raw_path) -> bool:
    """Search for movie*.json data files, then call retrieve_data method to download to the destination raw pool."""
    json_pattern = '^movie.*\.json$'
    for f in dbutils.fs.ls(source_path):
        if re.match(json_pattern, f.name) is not None:
            retrieve_data(f.name, dest_path)
            
prepare_raw()

### 1. Raw to Bronze

In [ ]:
from pyspark.sql.types import ArrayType, StructType,StructField, StringType, IntegerType, DateType, DoubleType, FloatType, LongType, TimestampType, DateType, BooleanType, MapType
from pyspark.sql.functions import col, explode, current_date, current_timestamp, lit, from_json, upper, lower, row_number, lag, last
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.window import Window

In [ ]:
# Prepare Bronze path
dbutils.fs.rm(bronze_path, recurse=True)

Out[8]: True

In [ ]:
# Ingest with a simple schema and metadata
movie_schema = StructType([StructField('movie', ArrayType(StringType()), True),])
    
def raw_to_bronze(src_path=raw_path, dest_path=bronze_path, schema=movie_schema, persist=True) -> DataFrame:
    """
    One by one ingestion to show json file names as the datasource.
    """
    for f in dbutils.fs.ls(src_path):
        movie_raw_df = (spark.read.option("inferSchema", 'false').option('multiline', 'true').schema(movie_schema).json(f.path))
        movie_raw_df = movie_raw_df.withColumn('movie', explode('movie'))
        movie_meta_df = movie_raw_df.select('Movie', lit(f.name).alias('SourceFile'), current_timestamp().alias('IngestTime'), current_timestamp().cast('date').alias('p_IngestDate'), lit('new').alias('Status'))
        if persist:
            movie_meta_df.write.format('delta').partitionBy('p_IngestDate').mode('append').save(dest_path)
        return movie_meta_df

movie_bronze = raw_to_bronze(persist=True)

In [ ]:
# Alternatively, batch ingestion with detailed schema
movie_schema = StructType().add('movie', ArrayType(
    StructType([
    StructField('BackdropUrl', StringType(), True),
    StructField('Budget', FloatType(), True),
    StructField('CreatedBy', StringType(), True),
    StructField('CreatedDate', TimestampType(), True),
    StructField('Id', LongType(), True),
    StructField('ImdbUrl', StringType(), True),
    StructField('OriginalLanguage', StringType(), True),
    StructField('Overview', StringType(), True),
    StructField('PosterUrl', StringType(), True),
    StructField('Price', FloatType(), True),
    StructField('ReleaseDate', TimestampType(), True),
    StructField('Revenue', FloatType(), True),
    StructField('RunTime', IntegerType(), True),
    StructField('Tagline', StringType(), True),
    StructField('Title', StringType(), True),
    StructField('TmdbUrl', StringType(), True),
    StructField('UpdatedBy', StringType(), True),
    StructField('UpdatedDate', TimestampType(), True),
    StructField('genres', ArrayType(StructType([
        StructField('id', LongType(), True),
        StructField('name', StringType(), True),
    ])), True),
])
), True)

def raw_to_bronze(src_path=raw_path, dest_path=bronze_path, schema=movie_schema, persist=True) -> DataFrame:
    """
    Batch ingestion with wildcard.
    """
    movie_raw_df = (spark.read.option("inferSchema", 'false').option('multiline', 'true').schema(movie_schema).json(src_path+'movie*.json'))
    movie_raw_df = movie_raw_df.withColumn('movie', explode('movie'))
    movie_meta_df = movie_raw_df.select('Movie', lit('movie_json').alias('SourceFile'), current_timestamp().alias('IngestTime'), current_timestamp().cast('date').alias('p_IngestDate'), lit('new').alias('Status'))
    if persist:
        movie_meta_df.write.format('delta').partitionBy('p_IngestDate').mode('append').save(dest_path)
    return movie_meta_df

movie_bronze = raw_to_bronze(persist=False)

In [ ]:
%sql

-- show history of the bronze folder
DESCRIBE HISTORY '/FileStore/movie/bronze'

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
0,2022-08-25T14:50:44.000+0000,1289947930359854,yokurt@yahoo.com,WRITE,"Map(mode -> Append, partitionBy -> [""p_IngestDate""])",null,List(936934055869446),0817-160429-43ys5vhs,null,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1249, numOutputBytes -> 522918)",null,Databricks-Runtime/10.4.x-scala2.12


In [ ]:
# Register the Bronze Table in the Metastore
spark.sql(
    """
    CREATE SCHEMA IF not EXISTS delta_lake
    """
)
spark.sql(
    """
    USE delta_lake
    """
)
spark.sql(
    """
    DROP TABLE IF EXISTS movie_bronze
    """
)
spark.sql(
    f"""
CREATE TABLE movie_bronze
USING DELTA
LOCATION "{bronze_path}"
"""
)

Out[10]: DataFrame[]

In [ ]:
%sql
-- query the bronze table
SELECT * FROM movie_bronze limit 10;

Movie,SourceFile,IngestTime,p_IngestDate,Status
"{""Id"":1,""Title"":""Inception"",""Overview"":""Cobb, a skilled thief who commits corporate espionage by infiltrating the subconscious of his targets is offered a chance to regain his old life as payment for a task considered to be impossible: \""inception\"", the implantation of another person's idea into a target's subconscious."",""Tagline"":""Your mind is the scene of the crime."",""Budget"":1.6E8,""Revenue"":8.25532764E8,""ImdbUrl"":""https://www.imdb.com/title/tt1375666"",""TmdbUrl"":""https://www.themoviedb.org/movie/27205"",""PosterUrl"":""https://image.tmdb.org/t/p/w342//9gk7adHYeDvHkCSEqAvQNLV5Uge.jpg"",""BackdropUrl"":""https://image.tmdb.org/t/p/original//s3TBrRGB1iav7gFOCNx3H31MoES.jpg"",""OriginalLanguage"":""en"",""ReleaseDate"":""2010-07-15T00:00:00"",""RunTime"":148,""Price"":9.9,""CreatedDate"":""2021-04-03T16:51:30.1633333"",""UpdatedDate"":null,""UpdatedBy"":null,""CreatedBy"":null,""genres"":[{""id"":1,""name"":""Adventure""},{""id"":6,""name"":""Action""},{""id"":13,""name"":""Science Fiction""}]}",movie_0.json,2022-08-25T05:59:46.822+0000,2022-08-25,new
"{""Id"":2,""Title"":""Interstellar"",""Overview"":""The adventures of a group of explorers who make use of a newly discovered wormhole to surpass the limitations on human space travel and conquer the vast distances involved in an interstellar voyage."",""Tagline"":""Mankind was born on Earth. It was never meant to die here."",""Budget"":1.65E8,""Revenue"":6.75120017E8,""ImdbUrl"":""https://www.imdb.com/title/tt0816692"",""TmdbUrl"":""https://www.themoviedb.org/movie/157336"",""PosterUrl"":""https://image.tmdb.org/t/p/w342//gEU2QniE6E77NI6lCU6MxlNBvIx.jpg"",""BackdropUrl"":""https://image.tmdb.org/t/p/original//xJHokMbljvjADYdit5fK5VQsXEG.jpg"",""OriginalLanguage"":""en"",""ReleaseDate"":""2014-11-05T00:00:00"",""RunTime"":169,""Price"":9.9,""CreatedDate"":""2021-04-03T16:51:30.1633333"",""UpdatedDate"":null,""UpdatedBy"":null,""CreatedBy"":null,""genres"":[{""id"":1,""name"":""Adventure""},{""id"":4,""name"":""Drama""},{""id"":13,""name"":""Science Fiction""}]}",movie_0.json,2022-08-25T05:59:46.822+0000,2022-08-25,new
"{""Id"":3,""Title"":""The Dark Knight"",""Overview"":""Batman raises the stakes in his war on crime. With the help of Lt. Jim Gordon and District Attorney Harvey Dent, Batman sets out to dismantle the remaining criminal organizations that plague the streets. The partnership proves to be effective, but they soon find themselves prey to a reign of chaos unleashed by a rising criminal mastermind known to the terrified citizens of Gotham as the Joker."",""Tagline"":""Why So Serious?"",""Budget"":1.85E8,""Revenue"":1.004558444E9,""ImdbUrl"":""https://www.imdb.com/title/tt0468569"",""TmdbUrl"":""https://www.themoviedb.org/movie/155"",""PosterUrl"":""https://image.tmdb.org/t/p/w342//qJ2tW6WMUDux911r6m7haRef0WH.jpg"",""BackdropUrl"":""https://image.tmdb.org/t/p/original//hkBaDkMWbLaf8B1lsWsKX7Ew3Xq.jpg"",""OriginalLanguage"":""en"",""ReleaseDate"":""2008-07-16T00:00:00"",""RunTime"":152,""Price"":9.9,""CreatedDate"":""2021-04-03T16:51:30.1633333"",""UpdatedDate"":null,""UpdatedBy"":null,""CreatedBy"":null,""genres"":[{""id"":4,""name"":""Drama""},{""id"":6,""name"":""Action""},{""id"":10,""name"":""Thriller""},{""id"":11,""name"":""Crime""}]}",movie_0.json,2022-08-25T05:59:46.822+0000,2022-08-25,new
"{""Id"":4,""Title"":""Deadpool"",""Overview"":""Deadpool tells the origin story of former Special Forces operative turned mercenary Wade Wilson, who after being subjected to a rogue experiment that leaves him with accelerated healing powers, adopts the alter ego Deadpool. Armed with his new abilities and a dark, twisted sense of humor, Deadpool hunts down the man who nearly destroyed his life."",""Tagline"":""Witness the beginning of a happy ending"",""Budget"":5.8E7,""Revenue"":7.831E8,""ImdbUrl"":""https://www.imdb.com/title/tt1431045"",""TmdbUrl"":""https://www.themoviedb.org/movie/293660"",""Poster

## 2. Bronze to Silver

### Checkout bronze data and unpack schema

In [ ]:
spark.sql(
    """
    USE delta_lake
    """
)

Out[1]: DataFrame[]

In [ ]:
# checkout bronze data
movie_bronze = spark.read.table("movie_bronze").filter("Status = 'new'").drop_duplicates()
movie_bronze.printSchema()

root
-- Movie: string (nullable = true)
-- SourceFile: string (nullable = true)
-- IngestTime: timestamp (nullable = true)
-- p_IngestDate: date (nullable = true)
-- Status: string (nullable = true)

In [ ]:
schema = StructType([
    StructField('Id', LongType(), True),
    StructField('Title', StringType(), True),
    StructField('Overview', StringType(), True),
    StructField('Tagline', StringType(), True),
    StructField('Budget', FloatType(), True),
    StructField('Revenue', FloatType(), True),
    StructField('ImdbUrl', StringType(), True),
    StructField('TmdbUrl', StringType(), True),
    StructField('PosterUrl', StringType(), True),
    StructField('BackdropUrl', StringType(), True),
    StructField('OriginalLanguage', StringType(), True),
    StructField('ReleaseDate', TimestampType(), True),
    StructField('RunTime', IntegerType(), True),
    StructField('Price', FloatType(), True),
    StructField('CreatedDate', TimestampType(), True),
    StructField('UpdatedDate', TimestampType(), True),
    StructField('CreatedBy', StringType(), True),
    StructField('UpdatedBy', StringType(), True),
    # StructField('genres', ArrayType(MapType(StringType(), StringType(), False)), True),
    StructField('genres', ArrayType(StructType([
        StructField('id', LongType(), True),
        StructField('name', StringType(), True),
    ])), True),
])
movie_augment_df = movie_bronze.withColumn('value', from_json(col('Movie'), schema)).select('value.*','*')
movie_augment_df = movie_augment_df.drop('value')
movie_augment_df.printSchema()

root
-- Id: long (nullable = true)
-- Title: string (nullable = true)
-- Overview: string (nullable = true)
-- Tagline: string (nullable = true)
-- Budget: float (nullable = true)
-- Revenue: float (nullable = true)
-- ImdbUrl: string (nullable = true)
-- TmdbUrl: string (nullable = true)
-- PosterUrl: string (nullable = true)
-- BackdropUrl: string (nullable = true)
-- OriginalLanguage: string (nullable = true)
-- ReleaseDate: timestamp (nullable = true)
-- RunTime: integer (nullable = true)
-- Price: float (nullable = true)
-- CreatedDate: timestamp (nullable = true)
-- UpdatedDate: timestamp (nullable = true)
-- CreatedBy: string (nullable = true)
-- UpdatedBy: string (nullable = true)
-- genres: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- id: long (nullable = true)
 | |-- name: string (nullable = true)
-- Movie: string (nullable = true)
-- SourceFile: string (nullable = true)
-- IngestTime: timestamp (nullable = true)
-- p_IngestDate: date (nullable = true)
-- Status: string (nullable = true)

### Generate junction and lookup tables

+ -- create language_silver delta and table
+ -- language table is special b/c **LangId** is newly generated by the process
+ -- other junction tables and lookup tables have predefined scopes by the source

In [ ]:
try:
    language_silver_ex = spark.read.table("language_silver")
    # language silver table exists
    # existing last/max language id in the silver table
    LangId_offset = (language_silver.orderBy(col('LangId').desc()).collect())[0]['LangId'] # int
    language_silver = movie_augment_df.select(col('OriginalLanguage').alias('Language')).drop_duplicates()
    language_silver = language_silver.subtract(language_silver_ex.select('Language')) # extract new languages
    language_silver = language_silver.withColumn('LangId', LangId_offset + row_number().over(Window.orderBy('Language'))) # assign new LangId
except:
    # language silver table not exist
    LangId_offset = 0
    language_silver = movie_augment_df.select(col('OriginalLanguage').alias('Language')).drop_duplicates()
    language_silver = language_silver.withColumn('LangId', row_number().over(Window.orderBy('Language')))

In [ ]:
movie_silver = movie_augment_df.join(language_silver, col('OriginalLanguage') == language_silver['language'], 'inner').drop('OriginalLanguage', 'language')
movie_silver = movie_silver.withColumn('genre', explode('genres')).select('*').drop('genres')

movie_genre_junction_silver = movie_silver.select(col('Id').alias('MovieId'), col('genre.id').alias('GenreId')).drop_duplicates() # no need for clean & quarantine
genre_silver = movie_silver.select(col('genre.id').alias('GenreId'), col('genre.name').alias('GenreName')).na.drop().drop_duplicates() # remove null genreName, no need for quarantine

### Cleanse and quarantine the bad

In [ ]:
# filter for genre missing
movie_qua_genre_missing = movie_silver.filter('genre.name is null')
# filter for low budget
movie_qua_budget = movie_silver.filter('Budget < 1000000')
# filter for negative runtime
movie_qua_runtime = movie_silver.filter('RunTime < 0')
# quarantine but leave the fixing to future 
movie_quarantine = movie_qua_genre_missing.union(movie_qua_budget).union(movie_qua_runtime).drop('genre').drop_duplicates() # 32 rows acquired

In [ ]:
movie_silver_clean = movie_silver.drop('genre').subtract(movie_quarantine) # 1249 - 32 = 1217 rows

In [ ]:
# add Status column for labeling
movie_quarantine = movie_quarantine.withColumn('Status', lit('quarantine'))
movie_silver_clean = movie_silver_clean.withColumn('Status', lit('load'))

### Persist

In [ ]:
language_silver.write.format('delta').mode('append').save(silver_path+'/language_silver')
movie_genre_junction_silver.write.format('delta').mode('append').save(silver_path+'/movie_genre_junction_silver')
genre_silver.write.format('delta').mode('append').save(silver_path+'/genre_silver')
movie_silver_clean.drop('Movie', 'Status').write.format('delta').mode('append').partitionBy('p_IngestDate').save(silver_path+'/movie_silver')
movie_quarantine.drop('Status').write.format('delta').mode('append').partitionBy('p_IngestDate').save(silver_path+'/movie_silver_quarantine')

In [ ]:
# spark.sql(
#     """
#     DROP TABLE IF EXISTS language_silver
#     """
# )

spark.sql(
    f"""
CREATE TABLE language_silver
USING DELTA
LOCATION "{silver_path}/language_silver"
"""
)

spark.sql(
    f"""
CREATE TABLE movie_genre_junction_silver
USING DELTA
LOCATION "{silver_path}/movie_genre_junction_silver"
"""
)

spark.sql(
    f"""
CREATE TABLE genre_silver
USING DELTA
LOCATION "{silver_path}/genre_silver"
"""
)

spark.sql(
    f"""
CREATE TABLE movie_silver
USING DELTA
LOCATION "{silver_path}/movie_silver"
"""
)

spark.sql(
    f"""
CREATE TABLE movie_silver_quarantine
USING DELTA
LOCATION "{silver_path}/movie_silver_quarantine"
"""
)

Out[129]: DataFrame[]

In [ ]:
%sql
-- query the movie_silver_quarantine table
SELECT * FROM movie_silver_quarantine limit 10;

Id,Title,Overview,Tagline,Budget,Revenue,ImdbUrl,TmdbUrl,PosterUrl,BackdropUrl,ReleaseDate,RunTime,Price,CreatedDate,UpdatedDate,CreatedBy,UpdatedBy,Movie,SourceFile,IngestTime,p_IngestDate,LangId
824,Paranormal Activity,"After a young, middle-class couple moves into what seems like a typical suburban house, they become increasingly disturbed by a presence that may or may not be demonic but is certainly the most active in the middle of the night. Followed by five terrifying installments in the franchise, this is the original found-footage shocker that started it all.",What Happens When You Sleep?,15000.0,1.93355808E8,https://www.imdb.com/title/tt1179904,https://www.themoviedb.org/movie/23827,https://image.tmdb.org/t/p/w342//1bjA7de4O0NhMsaOqwvrecophUs.jpg,https://image.tmdb.org/t/p/original//hxFjtYQ1YuAAGeK13yHE2ylu5gM.jpg,2009-09-25T00:00:00.000+0000,86,9.9,2021-04-03T16:51:30.296+0000,null,null,null,"{""Id"":824,""Title"":""Paranormal Activity"",""Overview"":""After a young, middle-class couple moves into what seems like a typical suburban house, they become increasingly disturbed by a presence that may or may not be demonic but is certainly the most active in the middle of the night. Followed by five terrifying installments in the franchise, this is the original found-footage shocker that started it all."",""Tagline"":""What Happens When You Sleep?"",""Budget"":15000.0,""Revenue"":1.933558E8,""ImdbUrl"":""https://www.imdb.com/title/tt1179904"",""TmdbUrl"":""https://www.themoviedb.org/movie/23827"",""PosterUrl"":""https://image.tmdb.org/t/p/w342//1bjA7de4O0NhMsaOqwvrecophUs.jpg"",""BackdropUrl"":""https://image.tmdb.org/t/p/original//hxFjtYQ1YuAAGeK13yHE2ylu5gM.jpg"",""OriginalLanguage"":""en"",""ReleaseDate"":""2009-09-25T00:00:00"",""RunTime"":86,""Price"":9.9,""CreatedDate"":""2021-04-03T16:51:30.2966667"",""UpdatedDate"":null,""UpdatedBy"":null,""CreatedBy"":null,""genres"":[{""id"":5,""name"":""Horror""},{""id"":14,""name"":""Mystery""}]}",movie_0.json,2022-08-25T21:29:49.947+0000,2022-08-25,1
1076,Isn't It Romantic,"For a long time, Natalie, an Australian architect living in New York City, had always believed that what she had seen in rom-coms is all fantasy. But after thwarting a mugger at a subway station only to be knocked out while fleeing, Natalie wakes up and discovers that her life has suddenly become her worst nightmare—a romantic comedy—and she is the leading lady.",None of the Feels,0.0,0.0,https://www.imdb.com/title/tt2452244,https://www.themoviedb.org/movie/449563,https://image.tmdb.org/t/p/w342//5xNBYXuv8wqiLVDhsfqCOr75DL7.jpg,https://image.tmdb.org/t/p/original//lGYlBtu16NswrgrSY1i3gZ1E1eI.jpg,2019-02-13T00:00:00.000+0000,89,9.9,2021-04-03T16:51:30.393+0000,null,null,null,"{""Id"":1076,""Title"":""Isn't It Romantic"",""Overview"":""For a long time, Natalie, an Australian architect living in New York City, had always believed that what she had seen in rom-coms is all fantasy. But after thwarting a mugger at a subway station only to be knocked out while fleeing, Natalie wakes up and discovers that her life has suddenly become her worst nightmare—a romantic comedy—and she is the leading lady."",""Tagline"":""None of the Feels"",""Budget"":0.0,""Revenue"":0.0,""ImdbUrl"":""https://www.imdb.com/title/tt2452244"",""TmdbUrl"":""https://www.themoviedb.org/movie/449563"",""PosterUrl"":""https://image.tmdb.org/t/p/w342//5xNBYXuv8wqiLVDhsfqCOr75DL7.jpg"",""BackdropUrl"":""https://image.tmdb.org/t/p/original//lGYlBtu16NswrgrSY1i3gZ1E1eI.jpg"",""OriginalLanguage"":""en"",""ReleaseDate"":""2019-02-13T00:00:00"",""RunTime"":89,""Price"":9.9,""CreatedDate"":""2021-04-03T16:51:30.3933333"",""UpdatedDate"":null,""UpdatedBy"":null,""CreatedBy"":null,""genres"":[{""id"":2,""name"":""Fantasy""},{""id"":7,""name"":""Comedy""},{""id"":16,""name"":""Romance""}]}",movie_0.json,2022-08-25T21:29:49.947+0000,2022-08-25,1
980,After We Collided,Tessa finds herself struggling with her complicated relationship with Hardin; she faces a d

### Lable the status in bronze

In [ ]:
# Create temp views
movie_silver_clean.createOrReplaceTempView('movie_silver_clean_tsv')
movie_quarantine.createOrReplaceTempView('movie_quarantine_silver_tsv')

In [ ]:
%sql
select * from `movie_silver_clean_tsv` limit 5

Id,Title,Overview,Tagline,Budget,Revenue,ImdbUrl,TmdbUrl,PosterUrl,BackdropUrl,ReleaseDate,RunTime,Price,CreatedDate,UpdatedDate,CreatedBy,UpdatedBy,Movie,SourceFile,IngestTime,p_IngestDate,Status,LangId
116,World War Z,"Life for former United Nations investigator Gerry Lane and his family seems content. Suddenly, the world is plagued by a mysterious infection turning whole human populations into rampaging mindless zombies. After barely escaping the chaos, Lane is persuaded to go on a mission to investigate this disease. What follows is a perilous trek around the world where Lane must brave horrific dangers and long odds to find answers before human civilization falls.",Remember Philly!,2.0E8,5.31864992E8,https://www.imdb.com/title/tt0816711,https://www.themoviedb.org/movie/72190,https://image.tmdb.org/t/p/w342//1SWBSYJsnyhdNRfLI1T6RsCxAQ4.jpg,https://image.tmdb.org/t/p/original//upsk7nfUjf8ZSYuokwa5U5YXERm.jpg,2013-06-20T00:00:00.000+0000,116,9.9,2021-04-03T16:51:30.190+0000,null,null,null,"{""Id"":116,""Title"":""World War Z"",""Overview"":""Life for former United Nations investigator Gerry Lane and his family seems content. Suddenly, the world is plagued by a mysterious infection turning whole human populations into rampaging mindless zombies. After barely escaping the chaos, Lane is persuaded to go on a mission to investigate this disease. What follows is a perilous trek around the world where Lane must brave horrific dangers and long odds to find answers before human civilization falls."",""Tagline"":""Remember Philly!"",""Budget"":2.0E8,""Revenue"":5.31865E8,""ImdbUrl"":""https://www.imdb.com/title/tt0816711"",""TmdbUrl"":""https://www.themoviedb.org/movie/72190"",""PosterUrl"":""https://image.tmdb.org/t/p/w342//1SWBSYJsnyhdNRfLI1T6RsCxAQ4.jpg"",""BackdropUrl"":""https://image.tmdb.org/t/p/original//upsk7nfUjf8ZSYuokwa5U5YXERm.jpg"",""OriginalLanguage"":""en"",""ReleaseDate"":""2013-06-20T00:00:00"",""RunTime"":116,""Price"":9.9,""CreatedDate"":""2021-04-03T16:51:30.1900000"",""UpdatedDate"":null,""UpdatedBy"":null,""CreatedBy"":null,""genres"":[{""id"":4,""name"":""Drama""},{""id"":5,""name"":""Horror""},{""id"":6,""name"":""Action""},{""id"":10,""name"":""Thriller""},{""id"":13,""name"":""Science Fiction""}]}",movie_0.json,2022-08-25T21:29:49.947+0000,2022-08-25,load,1
747,The Wizard of Oz,"Young Dorothy finds herself in a magical world where she makes friends with a lion, a scarecrow and a tin man as they make their way along the yellow brick road to talk with the Wizard and ask for the things they miss most in their lives. The Wicked Witch of the West is the only thing that could stop them.","We're off to see the Wizard, the wonderful Wizard of Oz!",2777000.0,3.3754968E7,https://www.imdb.com/title/tt0032138,https://www.themoviedb.org/movie/630,https://image.tmdb.org/t/p/w342//pfAZFD7I2hxW9HCChTuAzsdE6UX.jpg,https://image.tmdb.org/t/p/original//qAvou7F5P4VcIR72JzzrnKEQSN3.jpg,1939-08-15T00:00:00.000+0000,102,9.9,2021-04-03T16:51:30.283+0000,null,null,null,"{""Id"":747,""Title"":""The Wizard of Oz"",""Overview"":""Young Dorothy finds herself in a magical world where she makes friends with a lion, a scarecrow and a tin man as they make their way along the yellow brick road to talk with the Wizard and ask for the things they miss most in their lives. The Wicked Witch of the West is the only thing that could stop them."",""Tagline"":""We're off to see the Wizard, the wonderful Wizard of Oz!"",""Budget"":2777000.0,""Revenue"":3.3754967E7,""ImdbUrl"":""https://www.imdb.com/title/tt0032138"",""TmdbUrl"":""https://www.themoviedb.org/movie/630"",""PosterUrl"":""https://image.tmdb.org/t/p/w342//pfAZFD7I2hxW9HCChTuAzsdE6UX.jpg"",""BackdropUrl"":""https://image.tmdb.org/t/p/original//qAvou7F5P4VcIR72JzzrnKEQSN3.jpg"",""OriginalLanguage"":""en"",""ReleaseDate"":""1939-08-15T00:00:00"",""RunTime"":102,""Price"":9.9,""CreatedDate"":""2021-04-03T16:51:30.2833333"",""UpdatedDate"":null,""UpdatedBy"":null,""CreatedBy"":null,""genres"":

In [ ]:
# mark clean data as load
spark.sql(
    f"""
MERGE INTO movie_bronze
USING (
select Movie, SourceFile, IngestTime, p_IngestDate, Status
from movie_silver_clean_tsv 
) AS SOURCE 
ON movie_bronze.Movie = SOURCE.Movie
AND movie_bronze.IngestTime = SOURCE.IngestTime
WHEN MATCHED
THEN UPDATE
SET movie_bronze.Status = SOURCE.Status
"""
)

Out[88]: DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [ ]:
# mark quarantine data
spark.sql(
    f"""
MERGE INTO movie_bronze
USING (
select Movie, SourceFile, IngestTime, p_IngestDate, Status
from movie_quarantine_silver_tsv 
) AS SOURCE 
ON movie_bronze.Movie = SOURCE.Movie
AND movie_bronze.IngestTime = SOURCE.IngestTime
WHEN MATCHED
THEN UPDATE
SET movie_bronze.Status = SOURCE.Status
"""
)

Out[89]: DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [ ]:
%sql
select count(Movie) as Qty, first(Status) as Status from movie_bronze group by Status

Qty,Status
1217,load
32,quarantine


In [ ]:
%sql
drop table `movie_silver_clean_tsv`;
drop table `movie_quarantine_silver_tsv`;

## 3. Silver update